Blog here: https://wordpress.com/post/brianfarish3.wordpress.com/168

We are here: https://docs.aws.amazon.com/textract/latest/dg/api-async.html

Diagram of this pipeline: https://app.diagrams.net/#Hcodeslp%2Faws_arch_diagrams%2Fmaster%2Ftextract-stepfns.drawio

AWS has this which could be helpful to me in writing my code in the testract JSON to RDS Lambda.

boto3 textract documentation: 
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/textract.html#textract

AWS recommends using an asynchronous function to process multiple page files. I'm going to use step functions. More details on that decision in the blog.

#IAM
I am creating an IAM role for the Lambda function that will allow Step Functions to access resources. It uses the AWS standard policy AWSLambdaRole. The role will be named: step-functions-lanbda-role. The tag is "app": "document-parser"

#STEP FUNCTION
I'm creating a step function called Doc-Parser-Pipeline and attaching the above role to it. I'm not turning on logs now, but may later, as I inevitably mess up. Maybe even tracing to with X-ray. Tagged with app:document-parser.

State machine arn:  arn:aws:states:us-east-1:415832459288:stateMachine:Doc-Parser-Pipeline

#LAMBDA
I'm creating a step function "run-step-function-lambda" tagged with app:document-parser.

Adding environment variable:
STATEMACHINEARN:arn:aws:states:us-east-1:415832459288:stateMachine:Doc-Parser-Pipeline


#S3
Creating document upload bucket. Not public. Tag app:document-parser
Bucket name: 
doc-upload-bucket-doc-parser-app
S3 BUCKET ARN:
arn:aws:s3:::doc-upload-bucket-doc-parser-app

Properties > event notifications > create
Event name: 
trigger-document-parsing-event 

Prefix:
upload/

Suffix:
.pdf

Object creation, checking "all object create events". From the Lambda dropdown at the bottom selecting "run-step-functions-lambda"


# 2023/05/07 we are here


#LAMBDA

We are going to create three Lambdas ***BUT*** they will not actually have the logic we need in them yet. They are basically placeholders. These are in order that they are invoked in the step function:

Name:
doc-to-json-textract
ARN:
arn:aws:lambda:us-east-1:415832459288:function:doc-to-json-textract

Name:
textract-status-checker
ARN:
arn:aws:lambda:us-east-1:415832459288:function:textract-status-checker

Name:
parse-textract-json-obj-for-rds
ARN:
arn:aws:lambda:us-east-1:415832459288:function:parse-textract-json-obj-for-rds

#Reminder: At this point, all of these (and the initial run-step-functions-lambda also) are placeholders and contain no real logic.

#STEP FUNCTIONS

JSON will be pasted after this block.

Editing - Deleting the default Hello and World states from the "States" key. 
Generate code snippet > Invoke a function
Select fn from dropdown > doc-to-json-textract.

Click copy to copy the invoke JSON and paste into definition code snippet after "States": {. Click format JSON to clean it up.
Rename "Invoke function" to "Start extraction".  Make sure key above is "StartAt": "Start Extraction"

Change Next key to "Next:" "Wait for textract"

Add in Wait State. Change name to "Wait for Textract"

Generate code snippet > Invoke a function
Select fn from dropdown > textract-status-checker.  Change first line of new add from Invoke function to "Check textract status".  Make sure above key in Wait state is "Next": "Check textract status".

Add a code snippet for a Choice state. Will be edited heavily see code below. Don't need a "NOT" block. (The default state later will go back to the Wait state.) Essentially two blocks with "StringEquals": "COMPLETED" or second with "StringEquals": "FAILED". The variable "$.Payload.TextractJobStatus".

Change kv Default to "Default": "Wait for textract"

Generate code snippet > Invoke a function
Select fn from dropdown > parse-textract-json-obj-for-rds.  Change first line of new add from Invoke function to "Parse textract JSON".  Make sure above key in Wait state is "Next": "Parse textract JSON".

In choices COMPLETED block set kv "Next": "Parse textract JSON".

Change last "Next" to kv "End": true

For FAILED Choice block:
Generate snippet > flow control state > Fail state. Copy. Paste after END. Change as you see. Set FAILED block kv "Next": "Textract failed"

Chart here: 
/Users/bfaris96/Desktop/data_engineer/FCC-doc-tables/school_assessments_proj/Screen Shot 2023-05-08 at 3.59.30 PM.png


First pass at "run-step-function-lambda"

In [ ]:
import json
import boto3
import uuid
import os

def lambda_handler(event, context):
    step_function_client = boto3.client('stepfunctions')
    state_machine_arn = os.environ['STATEMACHINEARN']
    
    # Extract relevant information from the S3 event
    s3_bucket = event['Records'][0]['s3']['bucket']['name']
    s3_object_key = event['Records'][0]['s3']['object']['key']
    
    # Prepare the input for the state machine
    state_machine_input = {
        "s3Bucket": s3_bucket,
        "s3ObjectKey": s3_object_key
    }
    
    # Start the execution of the state machine
    response = step_function_client.start_execution(
        stateMachineArn=state_machine_arn,
        input=json.dumps(state_machine_input)
    )
    
    return {
        "statusCode": 200,
        "body": json.dumps({
            "message": "Step Function execution started.",
            "executionArn": response['executionArn']
        })
    }


First pass at "doc-to-json-textract"
# Just finished a draft of this cell below 2023-5-8.  Have not finished the cell above or any of the cells after .  Need to make sure we're being consistent on how you're using the step_state object and what you're passing in it.

In [ ]:
import json
import boto3

def lambda_handler(event, context):
    textract_client = boto3.client('textract')

    # Retrieve input from the state machine
    step_state = event['Input']
    s3_bucket = step_state['s3Bucket']
    s3_object_key = step_state['s3ObjectKey']

    #Parameters needed for the textract job:
    jobName = f'{s3_object_key}-{str(uuid.uuid4())}'.replace("/","-")
    all_text_key = f'textract-JSON-objects/{s3_object_key}-all-text.json'
    tables_key = f'textract-JSON-objects/{s3_object_key}-tables.json'

    # Start the Textract job for the uploaded document using StartDocumentTextDetection API for extracting raw text
    start_document_text_detection_response = textract_client.start_document_text_detection(
        DocumentLocation={
            'S3Object': {
                'Bucket': s3_bucket,
                'Name': f'{jobName}-all-text'
            }
        }
    )

    # Start the Textract job for the uploaded document using StartDocumentAnalysis API for extracting tables
    start_document_analysis_response = textract_client.start_document_analysis(
        DocumentLocation={
            'S3Object': {
                'Bucket': s3_bucket,
                'Name': f'{jobName}-tables'
            }
        },
        FeatureTypes=['TABLES']
    )

    # Return the Textract JobIds for the next Lambda functions to check the status
    step_state['all_text_key'] = all_text_key
    step_state['tables_key'] = tables_key
    step_state['textractJobName'] = jobName
    step_state['textDetectionJobId'] = start_document_text_detection_response['JobId']
    step_state['tableAnalysisJobId'] = start_document_analysis_response['JobId']

    
    return {
        "textDetectionJobId": start_document_text_detection_response['JobId'],
        "tableAnalysisJobId": start_document_analysis_response['JobId'],
        "s3Bucket": s3_bucket,
        "s3ObjectKey": s3_object_key
    }

First pass at "textract-status-checker"

In [ ]:
import json
import boto3

def lambda_handler(event, context):
    textract_client = boto3.client('textract')

    # Retrieve the Textract JobIds, S3 bucket, and object key from the input event
    text_detection_job_id = event['textDetectionJobId']
    table_analysis_job_id = event['tableAnalysisJobId']
    s3_bucket = event['s3Bucket']
    s3_object_key = event['s3ObjectKey']

    # Check the status of the raw text extraction job
    text_detection_status_response = textract_client.get_document_text_detection(JobId=text_detection_job_id)
    text_detection_status = text_detection_status_response['JobStatus']

    # Check the status of the table extraction job
    table_analysis_status_response = textract_client.get_document_analysis(JobId=table_analysis_job_id)
    table_analysis_status = table_analysis_status_response['JobStatus']

    # Return the status of both Textract jobs along with the S3 bucket and object key
    return {
        "textDetectionJobStatus": text_detection_status,
        "tableAnalysisJobStatus": table_analysis_status,
        "s3Bucket": s3_bucket,
        "s3ObjectKey": s3_object_key
    }
